In [53]:
import math

## Extinction and galactic latitude for given point

In [54]:
# Standard transform between magnitudes and nanoJanskies
def mag2flux(mag):
    # flux in nanoJ
    flux =  math.pow(10, (31.4 - mag)/2.5)
    return flux
def flux2mag(flux):
    # flux in nanoJ
    mag = 31.4 - 2.5*math.log10(flux)
    return mag

In [55]:
import sys
sys.path.append('/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/')
from dustmaps.sfd import SFDQuery
from astropy.coordinates import SkyCoord

### Choose an arbirary point in the sky

In [56]:
ra   = 270
decl = -30

### Calculate galactic latitude

In [57]:
# https://en.wikipedia.org/wiki/Galactic_coordinate_system
alphaNGP = 192.85948
deltaNGP =  27.1283
sdngp = math.sin(math.radians(deltaNGP))
cdngp = math.cos(math.radians(deltaNGP))
sde = math.sin(math.radians(decl))
cde = math.cos(math.radians(decl))
cra = math.cos(math.radians(ra - alphaNGP))
glat = math.degrees(math.asin(sdngp*sde + cdngp*cde*cra))
print('Galactic latitude is %.3f' % glat)

Galactic latitude is -3.236


### Calculate extinction
Need `pip install dustmaps`, which implements [Schlegel, Finkbeiner, and Davis](https://iopscience.iop.org/article/10.1086/305772)

In [58]:
sfd = SFDQuery()
c = SkyCoord(ra, decl, unit="deg", frame='icrs')
ebv = float(sfd(c))
print('E(B-V) %.2f' % ebv)

E(B-V) 0.92


### Corrected colour for extinction
These colour corrections are from Table 6 of 
[Schlafly and Finkbeiner](https://iopscience.iop.org/article/10.1088/0004-637X/737/2/103) with RV=3.1

In [59]:
# The LSST bands
EXTCOEF   = {'u':4.145, 'g':3.237, 'r':2.273, 'i':1.684, 'z':1.323, 'y':1.088}

In [60]:
# Modify magnituder for extinction
def dustmag(mag, band, ebv):
    return mag - ebv*EXTCOEF[band]

In [61]:
# Suppose we have a flux ratio of 1.4, meaning magnitude difference 0.365
fluxg = 100000.
fluxr = 140000.
gmag = flux2mag(fluxg)
rmag = flux2mag(fluxr)
print('g-r       %.3f' % (gmag-rmag))

gcorr = gmag - ebv*EXTCOEF['g']
rcorr = rmag - ebv*EXTCOEF['r']
print('g-r corr %.3f' % (gcorr-rcorr))

g-r       0.365
g-r corr -0.520
